In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import geopandas as gpd, geoviews as gv, geoplot as gplot
import warnings, random
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
polygons = gpd.read_file('station_polygons.geojson')
gplot.polyplot(polygons)
plt.show()

In [ ]:
stations = pd.read_table('Metadaten_Geographie_alle_stationen.txt', sep=';')
stations.rename(columns={'Stations_id': 'station_id', 'Stationsname': 'station_name', 'Stationshoehe': 'station_height',
                        'Geogr.Breite':'lat', 'Geogr.Laenge':'lon'},
               inplace=True)
stations['station_id'] = stations['station_id'].astype('str').str.rjust(5, '0')
stations = stations[stations['station_id'].isin(polygons['station_id'])]
stations = stations.groupby(['station_id', 'station_name'])[['station_height', 'lat', 'lon']].mean()
stations = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations['lat'], stations['lon']))
stations.head()

In [ ]:
weather = pd.read_table('weather_data.txt', sep=';')
weather['station_id'] = weather['station_id'].astype('str').str.rjust(5, '0')
weather.set_index(['station_id', 'measurement_date'], inplace=True)
weather.head()

# BootstrapClustGeo

In [ ]:
X = weather[['temperature_c', 'sunshine_h', 'precipitation_mm']]\
        [weather.index.get_level_values('measurement_date') == '2015-01-01'].\
        droplevel(1)
X

In [ ]:
X = stations.join(X, how='inner')